# 文件处理项目练习

## 解决问题

给定一个贷款信息数据，里面包括了贷款总金额（首付本金加贷款金额），贷款期限，年利率，可选首付金额等特征

amount,duration,rate,down_payment\
100000,36,0.08,20000\
200000,12,0.1,\
628400,120,0.12,100000\
4637400,240,0.06,\
42900,90,0.07,8900\
916000,16,0.13,\
45230,48,0.08,4300\
991360,99,0.08,\
423000,27,0.09,47200

根据上述的数据，计算出每行数据的EMI（等额本息），并在每行最后增加一个特征为EMI，每行是对应的EMI值

等额本息的公式为：
$EMI = P \times r \times \frac{(1 + r)^n}{(1 + r)^n - 1}$。



In [4]:
import math
# 首先写一个函数来计算等额本息

def loan_emi(amount, duration, rate, down_payment):
    """计算等额本息"""
    loan_amount = amount - down_payment  # loan_amount 表示贷款的金额
    emi  = (loan_amount * rate * (1 + rate) ** duration)  / ((1 + rate)**duration - 1)
    return math.ceil(emi)

loan_emi(100000, 36.0, 0.08, 2000)  # 测试代码

8364

## 数据的处理

In [5]:
with open('./data/loans1.txt', 'r') as f:
    loans1_file = f.readlines()

loans1_file

['amount,duration,rate,down_payment\n',
 '100000,36,0.08,20000\n',
 '200000,12,0.1,\n',
 '628400,120,0.12,100000\n',
 '4637400,240,0.06,\n',
 '42900,90,0.07,8900\n',
 '916000,16,0.13,\n',
 '45230,48,0.08,4300\n',
 '991360,99,0.08,\n',
 '423000,27,0.09,47200']

通过上述代码,我们能够看到,我们通过数据读取的方式,我们将数据一行一行的读取出来了

然后我们要根据头(特征)和体(特征值进行解析)

In [6]:
# 对特征进行解析
headers = loans1_file[0]
parse_headers = headers.strip().split(',')
parse_headers

['amount', 'duration', 'rate', 'down_payment']

将上述代码,封装到一个函数中

In [7]:
# 对特征进行解析的函数
def parse_headers(headers:str):
    return headers.strip().split(',')


In [8]:
parse_headers_result = parse_headers(loans1_file[0])
parse_headers_result

['amount', 'duration', 'rate', 'down_payment']

对特征值进行解析

In [9]:
result = []
for line in loans1_file[1:]:
    result.append(line.strip().split(','))

In [10]:
result

[['100000', '36', '0.08', '20000'],
 ['200000', '12', '0.1', ''],
 ['628400', '120', '0.12', '100000'],
 ['4637400', '240', '0.06', ''],
 ['42900', '90', '0.07', '8900'],
 ['916000', '16', '0.13', ''],
 ['45230', '48', '0.08', '4300'],
 ['991360', '99', '0.08', ''],
 ['423000', '27', '0.09', '47200']]

通过上述的代码, 会发现一个问题,有些数据里面没有`down_payment`特征值,怎么办

In [11]:
# 我们解析一行数据
values_1 = loans1_file[2]

values_1.strip().split(',')

['200000', '12', '0.1', '']

因为我们值都是需要参加运算的,我们希望所有的特征值都是浮点数, 所以我们尝试把一行数据的中每一个值拿出来转换为浮点数,包括为空的`down_payment`值 为0.0

In [12]:
parse_value_result = []
for value in values_1.strip().split(','):
    if value == '':
        parse_value_result.append(0.0)
    else:
        # 由于 非数字字符串是无法转换为浮点数,这里我们可以做一个异常处理
        try:
            parse_value_result.append(float(value))
        except ValueError:
            parse_value_result.append(value)

parse_value_result

[200000.0, 12.0, 0.1, 0.0]

In [13]:
# 将上述的代码进行封装
# 解析一行特征值的函数
def parse_value(value_line: str):
    parse_value = []
    for value in value_line.strip().split(','):
        if value == '':
            parse_value.append(0.0)
        else:
            # 由于 非数字字符串是无法转换为浮点数,这里我们可以做一个异常处理
            try:
                parse_value.append(float(value))
            except ValueError:
                parse_value.append(value)

    return parse_value

In [14]:
parse_value_result = parse_value(loans1_file[4])
parse_value_result

[4637400.0, 240.0, 0.06, 0.0]

紧接着,我们要对特征和特征值进行匹配

In [15]:
result = {}
for header, value in zip(parse_headers_result, parse_value_result):
    result[header] = value

result

{'amount': 4637400.0, 'duration': 240.0, 'rate': 0.06, 'down_payment': 0.0}

同样的方式,进行一个封装

In [16]:
def create_item_dict(headers, values):
    result = {}
    for header, value in zip(headers, values):
        result[header] = value

    return result  

In [17]:
create_item_dict(parse_headers_result, parse_value_result)

{'amount': 4637400.0, 'duration': 240.0, 'rate': 0.06, 'down_payment': 0.0}

通过上面的代码,我们得到了`parse_headers`:解析特征, 和`parse_value`:解析一行数据, 和`create_item_dict`函数:特征与特征值进行匹配,现在我们想得到多行数据的特征与特征值进行匹配,然后将匹配到的内容全部写入到数据容器中(列表)

In [18]:
def read_csv(path):
    result = []
    
    # 读取数据
    with open(path, 'r') as f:
        load_file = f.readlines()
        # print(load_file)  # 测试

    # 解析特征 
    headers = parse_headers(load_file[0])
    
    # 解析特征值
    for line_value in load_file[1:]:
        # 解析每一行特征值
        values = parse_value(line_value)

        # 将每一行特征值与特征进行匹配
        dict_header_vlaue = create_item_dict(headers, values)

        # 将匹配的结果添加到result里面
        result.append(dict_header_vlaue)

    return result

In [19]:
read_csv('./data/loans1.txt')

[{'amount': 100000.0, 'duration': 36.0, 'rate': 0.08, 'down_payment': 20000.0},
 {'amount': 200000.0, 'duration': 12.0, 'rate': 0.1, 'down_payment': 0.0},
 {'amount': 628400.0,
  'duration': 120.0,
  'rate': 0.12,
  'down_payment': 100000.0},
 {'amount': 4637400.0, 'duration': 240.0, 'rate': 0.06, 'down_payment': 0.0},
 {'amount': 42900.0, 'duration': 90.0, 'rate': 0.07, 'down_payment': 8900.0},
 {'amount': 916000.0, 'duration': 16.0, 'rate': 0.13, 'down_payment': 0.0},
 {'amount': 45230.0, 'duration': 48.0, 'rate': 0.08, 'down_payment': 4300.0},
 {'amount': 991360.0, 'duration': 99.0, 'rate': 0.08, 'down_payment': 0.0},
 {'amount': 423000.0, 'duration': 27.0, 'rate': 0.09, 'down_payment': 47200.0}]

通过上述的方式,就可以去读取csv文件的数据格式了

In [20]:
loans_result = read_csv('./data/loans1.txt')

In [21]:
loans_result

[{'amount': 100000.0, 'duration': 36.0, 'rate': 0.08, 'down_payment': 20000.0},
 {'amount': 200000.0, 'duration': 12.0, 'rate': 0.1, 'down_payment': 0.0},
 {'amount': 628400.0,
  'duration': 120.0,
  'rate': 0.12,
  'down_payment': 100000.0},
 {'amount': 4637400.0, 'duration': 240.0, 'rate': 0.06, 'down_payment': 0.0},
 {'amount': 42900.0, 'duration': 90.0, 'rate': 0.07, 'down_payment': 8900.0},
 {'amount': 916000.0, 'duration': 16.0, 'rate': 0.13, 'down_payment': 0.0},
 {'amount': 45230.0, 'duration': 48.0, 'rate': 0.08, 'down_payment': 4300.0},
 {'amount': 991360.0, 'duration': 99.0, 'rate': 0.08, 'down_payment': 0.0},
 {'amount': 423000.0, 'duration': 27.0, 'rate': 0.09, 'down_payment': 47200.0}]

In [22]:
for loan in loans_result:
    loan['emi'] = loan_emi(loan['amount'], loan['duration'], loan['rate'] / 12, loan['down_payment'])

In [23]:
loans_result

[{'amount': 100000.0,
  'duration': 36.0,
  'rate': 0.08,
  'down_payment': 20000.0,
  'emi': 2507},
 {'amount': 200000.0,
  'duration': 12.0,
  'rate': 0.1,
  'down_payment': 0.0,
  'emi': 17584},
 {'amount': 628400.0,
  'duration': 120.0,
  'rate': 0.12,
  'down_payment': 100000.0,
  'emi': 7582},
 {'amount': 4637400.0,
  'duration': 240.0,
  'rate': 0.06,
  'down_payment': 0.0,
  'emi': 33224},
 {'amount': 42900.0,
  'duration': 90.0,
  'rate': 0.07,
  'down_payment': 8900.0,
  'emi': 487},
 {'amount': 916000.0,
  'duration': 16.0,
  'rate': 0.13,
  'down_payment': 0.0,
  'emi': 62664},
 {'amount': 45230.0,
  'duration': 48.0,
  'rate': 0.08,
  'down_payment': 4300.0,
  'emi': 1000},
 {'amount': 991360.0,
  'duration': 99.0,
  'rate': 0.08,
  'down_payment': 0.0,
  'emi': 13712},
 {'amount': 423000.0,
  'duration': 27.0,
  'rate': 0.09,
  'down_payment': 47200.0,
  'emi': 15428}]

In [24]:
def compute_emis(loans):
    for loan in loans:
        loan['emi'] = loan_emi(
            loan['amount'], 
            loan['duration'], 
            loan['rate']/12, # CSV 包含年费率
            loan['down_payment'])

## 数据写入

In [25]:
with open('./data/emi1.txt', 'w') as f:
    for loan_result in loans_result:
        f.write('{},{},{},{},{}\n'.format(loan_result['amount'], 
                                          loan_result['duration'], 
                                          loan_result['rate'], 
                                          loan_result['down_payment'],
                                         loan_result['emi']))

In [26]:
!cat ./data/emi1.txt

100000.0,36.0,0.08,20000.0,2507
200000.0,12.0,0.1,0.0,17584
628400.0,120.0,0.12,100000.0,7582
4637400.0,240.0,0.06,0.0,33224
42900.0,90.0,0.07,8900.0,487
916000.0,16.0,0.13,0.0,62664
45230.0,48.0,0.08,4300.0,1000
991360.0,99.0,0.08,0.0,13712
423000.0,27.0,0.09,47200.0,15428


In [27]:
def write_csv(items, path):
    with open(path, 'w') as f:
        headers = list(items[0].keys())
        f.write(','.join(headers) + '\n')
        
        for item in items:
            values = []
            for header in headers:
                values.append(str(item.get(header, "")))
            f.write(','.join(values) + "\n")

In [28]:
loans3 = read_csv('./data/loans3.txt')

In [29]:
loans3

[{'amount': 45230.0, 'duration': 48.0, 'rate': 0.07, 'down_payment': 4300.0},
 {'amount': 883000.0, 'duration': 16.0, 'rate': 0.14, 'down_payment': 0.0},
 {'amount': 100000.0, 'duration': 12.0, 'rate': 0.1, 'down_payment': 0.0},
 {'amount': 728400.0,
  'duration': 120.0,
  'rate': 0.12,
  'down_payment': 100000.0},
 {'amount': 3637400.0, 'duration': 240.0, 'rate': 0.06, 'down_payment': 0.0},
 {'amount': 82900.0, 'duration': 90.0, 'rate': 0.07, 'down_payment': 8900.0},
 {'amount': 316000.0, 'duration': 16.0, 'rate': 0.13, 'down_payment': 0.0},
 {'amount': 15230.0, 'duration': 48.0, 'rate': 0.08, 'down_payment': 4300.0},
 {'amount': 991360.0, 'duration': 99.0, 'rate': 0.08, 'down_payment': 0.0},
 {'amount': 323000.0,
  'duration': 27.0,
  'rate': 0.09,
  'down_payment': 4720010000.0},
 {'amount': 528400.0,
  'duration': 120.0,
  'rate': 0.11,
  'down_payment': 100000.0},
 {'amount': 8633400.0, 'duration': 240.0, 'rate': 0.06, 'down_payment': 0.0},
 {'amount': 12900.0, 'duration': 90.0, '

In [30]:
compute_emis(loans3)

In [31]:
write_csv(loans3, './data/emis3.txt')

In [32]:
!cat ./data/emis3.txt

amount,duration,rate,down_payment,emi
45230.0,48.0,0.07,4300.0,981
883000.0,16.0,0.14,0.0,60819
100000.0,12.0,0.1,0.0,8792
728400.0,120.0,0.12,100000.0,9016
3637400.0,240.0,0.06,0.0,26060
82900.0,90.0,0.07,8900.0,1060
316000.0,16.0,0.13,0.0,21618
15230.0,48.0,0.08,4300.0,267
991360.0,99.0,0.08,0.0,13712
323000.0,27.0,0.09,4720010000.0,-193751447
528400.0,120.0,0.11,100000.0,5902
8633400.0,240.0,0.06,0.0,61853
12900.0,90.0,0.08,8900.0,60


In [33]:
loans_result

[{'amount': 100000.0,
  'duration': 36.0,
  'rate': 0.08,
  'down_payment': 20000.0,
  'emi': 2507},
 {'amount': 200000.0,
  'duration': 12.0,
  'rate': 0.1,
  'down_payment': 0.0,
  'emi': 17584},
 {'amount': 628400.0,
  'duration': 120.0,
  'rate': 0.12,
  'down_payment': 100000.0,
  'emi': 7582},
 {'amount': 4637400.0,
  'duration': 240.0,
  'rate': 0.06,
  'down_payment': 0.0,
  'emi': 33224},
 {'amount': 42900.0,
  'duration': 90.0,
  'rate': 0.07,
  'down_payment': 8900.0,
  'emi': 487},
 {'amount': 916000.0,
  'duration': 16.0,
  'rate': 0.13,
  'down_payment': 0.0,
  'emi': 62664},
 {'amount': 45230.0,
  'duration': 48.0,
  'rate': 0.08,
  'down_payment': 4300.0,
  'emi': 1000},
 {'amount': 991360.0,
  'duration': 99.0,
  'rate': 0.08,
  'down_payment': 0.0,
  'emi': 13712},
 {'amount': 423000.0,
  'duration': 27.0,
  'rate': 0.09,
  'down_payment': 47200.0,
  'emi': 15428}]

In [38]:
headers = list(loans_result[0].keys())

# values = [str(item[header]) if header in item else "" for header in headers]

for loan in loans_result:
    result = [str(loan[header]) if header in loan else "" for header in headers]
    print(result)
    result_str = ','.join(result)

print(result_str)

['100000.0', '36.0', '0.08', '20000.0', '2507']
['200000.0', '12.0', '0.1', '0.0', '17584']
['628400.0', '120.0', '0.12', '100000.0', '7582']
['4637400.0', '240.0', '0.06', '0.0', '33224']
['42900.0', '90.0', '0.07', '8900.0', '487']
['916000.0', '16.0', '0.13', '0.0', '62664']
['45230.0', '48.0', '0.08', '4300.0', '1000']
['991360.0', '99.0', '0.08', '0.0', '13712']
['423000.0', '27.0', '0.09', '47200.0', '15428']
423000.0,27.0,0.09,47200.0,15428
